In [ ]:
!kill -9 -1

In [1]:
from itertools import chain
import random
import numpy as np
import pandas as pd
import sklearn as sk
import sklearn.metrics
import scipy as sp
#import google.colab
#from google.colab import drive
import sklearn.linear_model
import tensorflow as tf
sess = tf.Session()
from keras import backend as K
from keras import *
from keras.layers import *
from keras.callbacks import LambdaCallback, Callback, ReduceLROnPlateau, TensorBoard
import sklearn.linear_model
#!pip install hyperas
from hyperas.distributions import uniform, choice
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
K.set_session(sess)
#drive.mount('/content/drive')


Using TensorFlow backend.


In [2]:
"""
Created on Tue Nov  6 10:06:52 2018

@author: yandexdataschool

Original Code found in:
https://github.com/yandexdataschool/roc_comparison

updated: Raul Sanchez-Vazquez
"""

import scipy.stats
from scipy import stats

# AUC comparison adapted from
# https://github.com/Netflix/vmaf/
def compute_midrank(x):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=np.float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2


def compute_midrank_weight(x, sample_weight):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    cumulative_weight = np.cumsum(sample_weight[J])
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = cumulative_weight[i:j].mean()
        i = j
    T2 = np.empty(N, dtype=np.float)
    T2[J] = T
    return T2


def fastDeLong(predictions_sorted_transposed, label_1_count, sample_weight):
    if sample_weight is None:
        return fastDeLong_no_weights(predictions_sorted_transposed, label_1_count)
    else:
        return fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight)


def fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank_weight(positive_examples[r, :], sample_weight[:m])
        ty[r, :] = compute_midrank_weight(negative_examples[r, :], sample_weight[m:])
        tz[r, :] = compute_midrank_weight(predictions_sorted_transposed[r, :], sample_weight)
    total_positive_weights = sample_weight[:m].sum()
    total_negative_weights = sample_weight[m:].sum()
    pair_weights = np.dot(sample_weight[:m, np.newaxis], sample_weight[np.newaxis, m:])
    total_pair_weights = pair_weights.sum()
    aucs = (sample_weight[:m]*(tz[:, :m] - tx)).sum(axis=1) / total_pair_weights
    v01 = (tz[:, :m] - tx[:, :]) / total_negative_weights
    v10 = 1. - (tz[:, m:] - ty[:, :]) / total_positive_weights
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def fastDeLong_no_weights(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating
              Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.
    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)
    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
    return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


def compute_ground_truth_statistics(ground_truth, sample_weight):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    if sample_weight is None:
        ordered_sample_weight = None
    else:
        ordered_sample_weight = sample_weight[order]

    return order, label_1_count, ordered_sample_weight


def delong_roc_variance(ground_truth, predictions, sample_weight=None):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    order, label_1_count, ordered_sample_weight = compute_ground_truth_statistics(
        ground_truth, sample_weight)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count, ordered_sample_weight)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov


In [3]:
def data():


    import pandas as pd

    column_names = ['A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','A11','A12','A13','A14','A15','A16']
    converters = {'A1': str
                  ,'A2': np.float64
                  ,'A3': np.float64
                  ,'A4': str
                  ,'A5': str
                  ,'A6': str
                  ,'A7': str
                  ,'A8': np.float64
                  ,'A9': str
                  ,'A10': str
                  ,'A11': np.float64
                  ,'A12': str
                  ,'A13': str
                  ,'A14': np.float64
                  ,'A15': np.float64
                  ,'A16': str}

    credit = pd.read_csv(
        "~/Google Drive/Discrétisation ICLR19/opendata/credit-screening.data",
        sep=",",
        header= None,
        names = column_names,
        na_values=[
            '-NaN', '-nan',
            'N/A', 'NA', 'NULL', 'NaN', 'n/a', 'nan', 'null',
            '.'
        ])    
    
    credit.dropna(inplace=True)
    credit.A16 = sklearn.preprocessing.LabelEncoder().fit_transform(credit.A16)
    credit.reset_index(inplace=True, drop=True)
    lignes = np.any(credit.astype(str)=='?',axis=1)
    credit.drop(credit.index[lignes], axis=0, inplace=True)
    credit.reset_index(inplace=True, drop=True)
    
    credit_label_encoders = []

    credit_encoded = credit.copy()

    for j in ['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']:
        temp = sk.preprocessing.LabelEncoder()
        temp.fit(credit[j].astype(str))
        credit_label_encoders.append(temp)
        credit_encoded[j] = temp.transform(credit[j].astype(str))
        
    credit_one_hot_encoder = sk.preprocessing.OneHotEncoder(categories='auto',sparse=False,handle_unknown="ignore")
    credit_one_hot_encoder.fit(credit_encoded[[
    'A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13'
    ]])
    credit_one_hot_encoded = credit_encoded.copy()
    credit_one_hot_encoded.drop(
    ['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13'],
    axis=1,
    inplace=True)
    credit_one_hot_encoded = pd.concat(
    [
    credit_one_hot_encoded,
    pd.DataFrame(
    credit_one_hot_encoder.transform(credit_encoded[[
      'A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13'
    ]]),
    index=credit_one_hot_encoded.index)
    ],
    axis=1)  
    credit_features_train, credit_features_test, credit_perf_train, credit_perf_test = sk.model_selection.train_test_split(
    credit_one_hot_encoded.drop('A16', axis=1),
    credit_one_hot_encoded.A16,
    test_size=0.33,
    random_state=1)
    credit_nn_features_train = credit_encoded.iloc[
    credit_features_train.index, :].drop(
    'A16', axis=1)
    credit_nn_features_test = credit_encoded.iloc[credit_features_test.index, :].drop(
    'A16', axis=1)
    credit_nn_perf_train = credit_encoded.iloc[
    credit_features_train.index, :].A16
    credit_nn_perf_test = credit_encoded.iloc[credit_features_test.index, :].A16

    
    global n
    global n_test
    n = credit_nn_features_train.shape[0]
    n_test = credit_nn_features_test.shape[0]



    x_quant = credit_nn_features_train[[
    'A2','A3','A8','A11','A14','A15'
    ]].values
    x_qual = credit_nn_features_train[['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']].values
    y = credit_nn_perf_train.values


    d1 = x_quant.shape[1]
    d2 = x_qual.shape[1]





    x_quant_test = credit_nn_features_test[[
    'A2','A3','A8','A11','A14','A15'
    ]].values
    x_qual_test = credit_nn_features_test[['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']].values
    y_test = credit_nn_perf_test.values


    for j in range(d2):
        liste_absents = [
        item for item in np.unique(x_qual_test[:, j])
        if item not in np.unique(x_qual[:, j])
        ]
        liste_lignes = [
        np.where(x_qual_test[:, j] == item) for item in liste_absents
        ]
        x_qual_test = np.delete(
        x_qual_test,
        list(chain.from_iterable(list(chain.from_iterable(liste_lignes)))),
        axis=0)
        x_quant_test = np.delete(
        x_quant_test,
        list(chain.from_iterable(list(chain.from_iterable(liste_lignes)))),
        axis=0)
        y_test = np.delete(
        y_test,
        list(chain.from_iterable(list(chain.from_iterable(liste_lignes)))),
        axis=0)
    
    x_quant = x_quant.astype(float)
    x_quant_test = x_quant_test.astype(float)
    
    x_qual_encoded = x_qual.copy()
    x_qual_label_encoders = []
    
    for j in range(d2):
        temp = sk.preprocessing.LabelEncoder()
        temp.fit(x_qual[:,j].astype(str))
        x_qual_label_encoders.append(temp)
        x_qual_encoded[:,j] = temp.transform(x_qual[:,j].astype(str))
        
    x_qual_one_hot_encoder = sk.preprocessing.OneHotEncoder(categories='auto',sparse=False,handle_unknown="ignore")
    
    x_qual_one_hot_encoder.fit(x_qual_encoded)
    
    x_qual_dummy = x_qual_one_hot_encoder.transform(x_qual_encoded)
    
    x_qual = x_qual_encoded
    
    x_qual_test_encoded = x_qual_test.copy()
    
    for j in range(d2):
        x_qual_test_encoded[:,j] = x_qual_label_encoders[j].transform(x_qual_test[:,j].astype(str))
        
    x_qual_test = x_qual_test_encoded

    x_qual_dummy_test = x_qual_one_hot_encoder.transform(x_qual_test)

    
    return x_quant,x_qual,x_qual_dummy,y,x_quant_test,x_qual_test,x_qual_dummy_test,y_test


In [4]:
x_quant,x_qual,x_qual_dummy,y,x_quant_test,x_qual_test,x_qual_dummy_test,y_test = data()

In [5]:
np.savetxt("x_quant.csv", x_quant, delimiter=",")
np.savetxt("x_qual.csv", x_qual, delimiter=",")
np.savetxt("y.csv", y, delimiter=",")
np.savetxt("x_quant_test.csv", x_quant_test, delimiter=",")
np.savetxt("x_qual_test.csv", x_qual_test, delimiter=",")
np.savetxt("y_test.csv", y_test, delimiter=",")

In [4]:
n = 437
n_test = 212
d1 = 6
d2 = 9

liste_layers_quant = [None] * d1
liste_layers_qual = [None] * d2
liste_qual_arrays = [None] * d2
liste_qual_arrays_test = [None] * d2

def create_model(x_quant,x_qual,x_qual_dummy,y,x_quant_test,x_qual_test,x_qual_dummy_test,y_test):    
    def initialize_neural_net(m_quant,m_qual):
        liste_inputs_quant = [None] * d1
        liste_inputs_qual = [None] * d2

        liste_layers_quant = [None] * d1
        liste_layers_qual = [None] * d2

        liste_layers_quant_inputs = [None] * d1
        liste_layers_qual_inputs = [None] * d2

        for i in range(d1):
            liste_inputs_quant[i] = Input((1, ))
            liste_layers_quant[i] = Dense(m_quant[i], activation='softmax')
            liste_layers_quant_inputs[i] = liste_layers_quant[i](
                liste_inputs_quant[i])

        for i in range(d2):
            liste_inputs_qual[i] = Input((len(np.unique(x_qual[:, i])), ))
            if (len(np.unique(x_qual[:, i])) > m_qual[i]):
                liste_layers_qual[i] = Dense(
                m_qual[i], activation='softmax', use_bias=False)
            else:
                liste_layers_qual[i] = Dense(
                len(np.unique(x_qual[:, i])), activation='softmax', use_bias=False)

            liste_layers_qual_inputs[i] = liste_layers_qual[i](
                liste_inputs_qual[i])

        return ([
            liste_inputs_quant, liste_layers_quant, liste_layers_quant_inputs,
            liste_inputs_qual, liste_layers_qual, liste_layers_qual_inputs
        ])
    
    
    def from_layers_to_proba_training(d1,d2,liste_layers_quant,liste_layers_qual):

        results = [None] * (d1 + d2)

        for j in range(d1):
            results[j] = K.function([liste_layers_quant[j].input],
                                    [liste_layers_quant[j].output])(
                                        [x_quant[:, j, np.newaxis]])

        for j in range(d2):
            results[j + d1] = K.function([liste_layers_qual[j].input],
                                         [liste_layers_qual[j].output])(
                                             [liste_qual_arrays[j]])

        return (results)
    
    
    
    
    def from_weights_to_proba_test(d1,d2,m_quant,m_qual,history,x_quant_test,x_qual_test,n_test):

        results = [None] * (d1 + d2)    

        for j in range(d1):
            results[j] = np.zeros((n_test,m_quant[j]))
            for i in range(m_quant[j]):
                results[j][:,i] = history.best_weights[j][1][i] + history.best_weights[j][0][0][i]*x_quant_test[:,j]


        for j in range(d2):
            results[j+d1] = np.zeros((n_test,history.best_weights[j+d1][0].shape[1]))
            for i in range(history.best_weights[j+d1][0].shape[1]):
                for k in range(n_test):
                    results[j+d1][k,i] = history.best_weights[j+d1][0][x_qual_test[k,j],i]

        return(results)
    
    
    def evaluate_disc(type,d1,d2,misc):
        if type=="train":
            proba = from_layers_to_proba_training(d1,d2,misc[0],misc[1])
        else:
            proba = from_weights_to_proba_test(d1,d2,misc[0],misc[1],misc[2],misc[3],misc[4],misc[5])


        results = [None] * (d1 + d2)

        if type=="train":
            X_transformed = np.ones((n, 1))
        else:
            X_transformed = np.ones((n_test, 1))

        for j in range(d1 + d2):
            if type=="train":
                results[j] = np.argmax(proba[j][0], axis=1)
            else:
                results[j] = np.argmax(proba[j], axis=1)
            
            X_new = sk.preprocessing.OneHotEncoder(categories = "auto", sparse=False, handle_unknown = "ignore").fit_transform(
                    X=results[j].reshape(-1, 1))
            if X_new.shape[1] > 1:
                X_transformed = np.concatenate((X_transformed, X_new),axis=1)

        proposed_logistic_regression = sk.linear_model.LogisticRegression(
            fit_intercept=False, solver = "lbfgs", C=1e20, tol=1e-8)


        if type=="train":
            proposed_logistic_regression.fit(X=X_transformed, y=y.reshape((n, )))
            performance = 2 * sk.metrics.log_loss(
              y,
              proposed_logistic_regression.predict_proba(X=X_transformed)[:, 1],
              normalize=False
          ) + proposed_logistic_regression.coef_.shape[1] * np.log(n)
            predicted = proposed_logistic_regression.predict_proba(X_transformed)[:,1]

        else:
            proposed_logistic_regression.fit(X=X_transformed, y=y_test.reshape((n_test, )))
            performance = 2*sk.metrics.roc_auc_score(y_test,proposed_logistic_regression.predict_proba(X_transformed)[:,1])-1
            predicted = proposed_logistic_regression.predict_proba(X_transformed)[:,1]

        return (performance, predicted)


    
    class LossHistory(Callback):
        def on_train_begin(self, logs={}):
            self.losses = []
            self.best_criterion = float("inf")
            self.best_outputs = []

        def on_epoch_end(self, batch, logs={}):
            self.losses.append(evaluate_disc("train",d1,d2,[liste_layers_quant,liste_layers_qual])[0])
            if self.losses[-1] < self.best_criterion:
                self.best_weights = []
                self.best_outputs = []
                self.best_criterion = self.losses[-1]
                for j in range(d1):
                    self.best_weights.append(liste_layers_quant[j].get_weights())
                    self.best_outputs.append(
                        K.function([liste_layers_quant[j].input],
                                   [liste_layers_quant[j].output])(
                                       [x_quant[:, j, np.newaxis]]))
                for j in range(d2):
                    self.best_weights.append(liste_layers_qual[j].get_weights())
                    self.best_outputs.append(
                        K.function([liste_layers_qual[j].input],
                                   [liste_layers_qual[j].output])(
                                       [liste_qual_arrays[j]]))
    
    
    
    essai = {{uniform(4,9)}}

    m_quant = [int(essai)] * d1
    m_qual = [int(essai)] * d2

    global liste_qual_arrays
    liste_qual_arrays = [None] * d2
    cursor = 0
    for j in range(d2):
        liste_qual_arrays[j] = x_qual_dummy[:, cursor:(
            cursor + len(np.unique(x_qual[:, j])))]
        cursor += len(np.unique(x_qual[:, j]))

    global liste_qual_arrays_test
    liste_qual_arrays_test = [None] * d2
    cursor = 0
    for j in range(d2):
        liste_qual_arrays_test[j] = x_qual_dummy_test[:, cursor:(
            cursor + len(np.unique(x_qual_test[:, j])))]
        cursor += len(np.unique(x_qual_test[:, j]))
  
  
    liste_inputs_quant = [None] * d1
    liste_inputs_qual = [None] * d2

    global liste_layers_quant
    global liste_layers_qual

    liste_layers_quant = [None] * d1
    liste_layers_qual = [None] * d2

    liste_layers_quant_inputs = [None] * d1
    liste_layers_qual_inputs = [None] * d2

    for i in range(d1):
        liste_inputs_quant[i] = Input((1, ))
        liste_layers_quant[i] = Dense(m_quant[i], activation='softmax')
        liste_layers_quant_inputs[i] = liste_layers_quant[i](
          liste_inputs_quant[i])

    for i in range(d2):
        liste_inputs_qual[i] = Input((len(np.unique(x_qual[:, i])), ))
        if (len(np.unique(x_qual[:, i])) > m_qual[i]):
            liste_layers_qual[i] = Dense(
            m_qual[i], activation='softmax', use_bias=False)
        else:
            liste_layers_qual[i] = Dense(
            len(np.unique(x_qual[:, i])), activation='softmax', use_bias=False)

        liste_layers_qual_inputs[i] = liste_layers_qual[i](
        liste_inputs_qual[i])


    full_hidden = concatenate(
        list(
          chain.from_iterable(
              [liste_layers_quant_inputs, liste_layers_qual_inputs])))
    output = Dense(1, activation='sigmoid')(full_hidden)
    model = Model(
      inputs=list(chain.from_iterable([liste_inputs_quant, liste_inputs_qual])),
      outputs=[output])  

    
    adam = optimizers.Adam(lr={{choice([10**-3, 10**-2, 10**-1])}})
    rmsprop = optimizers.RMSprop(lr={{choice([10**-3, 10**-2, 10**-1])}})
    sgd = optimizers.SGD(lr={{choice([10**-3, 10**-2, 10**-1])}})
   
    choiceval = {{choice(['adam', 'sgd', 'rmsprop'])}}
    if choiceval == 'adam':
        optim = adam
    elif choiceval == 'rmsprop':
        optim = rmsprop
    else:
        optim = sgd
        
 
    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])

    history = LossHistory()

    callbacks = [
      ReduceLROnPlateau(
          monitor='loss',
          factor=0.5,
          patience=10,
          verbose=0,
          mode='auto',
          min_delta=0.0001,
          cooldown=0,
          min_lr=0), history
    ]
    model.fit(
      list(chain.from_iterable([list(x_quant.T), liste_qual_arrays])),
      y,
      epochs=300,
      batch_size={{choice([32,64,128])}},
      verbose=1,
      callbacks=callbacks)

    global n_test
    n_test = x_quant_test.shape[0]
    performance, predicted = evaluate_disc("test",d1,d2,misc=[m_quant,m_qual,history,x_quant_test,x_qual_test,n_test])
  
    K.clear_session()
    return {'loss': -performance, 'status': STATUS_OK, 'model': model, 'predicted': predicted}


In [5]:
trials = Trials()

best_run, best_model, space = optim.minimize(model=create_model,
                          data=data,
                          algo=tpe.suggest,
                          max_evals=10,
                          trials=trials,
                          notebook_name='hyperas - credit-screening - dump - ICML19',
                          eval_space=True,
                          return_space=True)

>>> Imports:
#coding=utf-8

try:
    from itertools import chain
except:
    pass

try:
    import random
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import sklearn as sk
except:
    pass

try:
    import sklearn.metrics
except:
    pass

try:
    import scipy as sp
except:
    pass

try:
    import sklearn.linear_model
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    from keras import *
except:
    pass

try:
    from keras.layers import *
except:
    pass

try:
    from keras.callbacks import LambdaCallback, Callback, ReduceLROnPlateau, TensorBoard
except:
    pass

try:
    import sklearn.linear_model
except:
    pass

try:
    from hyperas.distributions import uniform, choice
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
 

Epoch 1/300
437/437 [==============================] - 2s 4ms/step - loss: 0.6738 - acc: 0.5744
(437, 1)
(437, 1)
(437, 2)
(437, 1)
(437, 2)
(437, 3)
(437, 2)
(437, 5)
(437, 2)
(437, 7)
(437, 2)
(437, 9)
(437, 2)
(437, 11)
(437, 3)
(437, 13)
(437, 2)
(437, 16)
(437, 7)
(437, 18)
(437, 4)
(437, 25)
(437, 1)
(437, 29)
(437, 2)
(437, 29)
(437, 1)
(437, 31)
(437, 3)
(437, 31)


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Epoch 2/300
437/437 [==============================] - 0s 79us/step - loss: 0.6501 - acc: 0.7025
(437, 1)
(437, 1)
(437, 2)
(437, 1)
(437, 3)
(437, 3)
(437, 2)
(437, 6)
(437, 2)
(437, 8)
(437, 2)
(437, 10)
(437, 2)
(437, 12)
(437, 3)
(437, 14)
(437, 2)
(437, 17)
(437, 7)
(437, 19)
(437, 4)
(437, 26)
(437, 1)
(437, 30)
(437, 2)
(437, 30)
(437, 1)
(437, 32)
(437, 3)
(437, 32)


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


...

437/437 [==============================] - 0s 44us/step - loss: 0.2797 - acc: 0.8764
(437, 1)
(437, 1)
(437, 2)
(437, 1)
(437, 2)
(437, 3)
(437, 2)
(437, 5)
(437, 2)
(437, 7)
(437, 1)
(437, 9)
(437, 2)
(437, 9)
(437, 2)
(437, 11)
(437, 3)
(437, 13)
(437, 4)
(437, 16)
(437, 3)
(437, 20)
(437, 2)
(437, 23)
(437, 2)
(437, 25)
(437, 2)
(437, 27)
(437, 2)
(437, 29)
Epoch 210/300
437/437 [==============================] - 0s 50us/step - loss: 0.2797 - acc: 0.8787
(437, 1)
(437, 1)
(437, 2)
(437, 1)
(437, 2)
(437, 3)
(437, 2)
(437, 5)
(437, 2)
(437, 7)
(437, 1)
(437, 9)
(437, 2)
(437, 9)
(437, 2)
(437, 11)
(437, 3)
(437, 13)
(437, 4)
(437, 16)
(437, 3)
(437, 20)
(437, 2)
(437, 23)
(437, 2)
(437, 25)
(437, 2)
(437, 27)
(437, 2)
(437, 29)
Epoch 211/300
437/437 [==============================] - 0s 53us/step - loss: 0.2795 - acc: 0.8764
(437, 1)
(437, 1)
(437, 2)
(437, 1)
(437, 2)
(437, 3)
(437, 2)
(437, 5)
(437, 2)
(437, 7)
(437, 1)
(437, 9)
(437, 2)
(437, 9)
(437, 2)
(437, 11)
(437, 3)
(437, 1

437/437 [==============================] - 0s 41us/step - loss: 0.2769 - acc: 0.8810
(437, 1)
(437, 1)
(437, 2)
(437, 1)
(437, 2)
(437, 3)
(437, 2)
(437, 5)
(437, 2)
(437, 7)
(437, 1)
(437, 9)
(437, 2)
(437, 9)
(437, 2)
(437, 11)
(437, 3)
(437, 13)
(437, 4)
(437, 16)
(437, 3)
(437, 20)
(437, 2)
(437, 23)
(437, 2)
(437, 25)
(437, 2)
(437, 27)
(437, 2)
(437, 29)
Epoch 232/300
437/437 [==============================] - 0s 44us/step - loss: 0.2768 - acc: 0.8833
(437, 1)
(437, 1)
(437, 2)
(437, 1)
(437, 2)
(437, 3)
(437, 2)
(437, 5)
(437, 2)
(437, 7)
(437, 1)
(437, 9)
(437, 2)
(437, 9)
(437, 2)
(437, 11)
(437, 3)
(437, 13)
(437, 4)
(437, 16)
(437, 3)
(437, 20)
(437, 2)
(437, 23)
(437, 2)
(437, 25)
(437, 2)
(437, 27)
(437, 2)
(437, 29)
Epoch 233/300
437/437 [==============================] - 0s 51us/step - loss: 0.2767 - acc: 0.8833
(437, 1)
(437, 1)
(437, 2)
(437, 1)
(437, 2)
(437, 3)
(437, 2)
(437, 5)
(437, 2)
(437, 7)
(437, 1)
(437, 9)
(437, 2)
(437, 9)
(437, 2)
(437, 11)
(437, 3)
(437, 1

437/437 [==============================] - 0s 42us/step - loss: 0.2761 - acc: 0.8787
(437, 1)
(437, 1)
(437, 2)
(437, 1)
(437, 2)
(437, 3)
(437, 2)
(437, 5)
(437, 2)
(437, 7)
(437, 1)
(437, 9)
(437, 2)
(437, 9)
(437, 2)
(437, 11)
(437, 3)
(437, 13)
(437, 4)
(437, 16)
(437, 3)
(437, 20)
(437, 2)
(437, 23)
(437, 2)
(437, 25)
(437, 2)
(437, 27)
(437, 2)
(437, 29)
Epoch 254/300
437/437 [==============================] - 0s 41us/step - loss: 0.2761 - acc: 0.8787
(437, 1)
(437, 1)
(437, 2)
(437, 1)
(437, 2)
(437, 3)
(437, 2)
(437, 5)
(437, 2)
(437, 7)
(437, 1)
(437, 9)
(437, 2)
(437, 9)
(437, 2)
(437, 11)
(437, 3)
(437, 13)
(437, 4)
(437, 16)
(437, 3)
(437, 20)
(437, 2)
(437, 23)
(437, 2)
(437, 25)
(437, 2)
(437, 27)
(437, 2)
(437, 29)
Epoch 255/300
437/437 [==============================] - 0s 42us/step - loss: 0.2761 - acc: 0.8787
(437, 1)
(437, 1)
(437, 2)
(437, 1)
(437, 2)
(437, 3)
(437, 2)
(437, 5)
(437, 2)
(437, 7)
(437, 1)
(437, 9)
(437, 2)
(437, 9)
(437, 2)
(437, 11)
(437, 3)
(437, 1

437/437 [==============================] - 0s 43us/step - loss: 0.2757 - acc: 0.8764
(437, 1)
(437, 1)
(437, 2)
(437, 1)
(437, 2)
(437, 3)
(437, 2)
(437, 5)
(437, 2)
(437, 7)
(437, 1)
(437, 9)
(437, 2)
(437, 9)
(437, 2)
(437, 11)
(437, 3)
(437, 13)
(437, 4)
(437, 16)
(437, 3)
(437, 20)
(437, 2)
(437, 23)
(437, 2)
(437, 25)
(437, 2)
(437, 27)
(437, 2)
(437, 29)
Epoch 276/300
437/437 [==============================] - 0s 43us/step - loss: 0.2756 - acc: 0.8787
(437, 1)
(437, 1)
(437, 2)
(437, 1)
(437, 2)
(437, 3)
(437, 2)
(437, 5)
(437, 2)
(437, 7)
(437, 1)
(437, 9)
(437, 2)
(437, 9)
(437, 2)
(437, 11)
(437, 3)
(437, 13)
(437, 4)
(437, 16)
(437, 3)
(437, 20)
(437, 2)
(437, 23)
(437, 2)
(437, 25)
(437, 2)
(437, 27)
(437, 2)
(437, 29)
Epoch 277/300
437/437 [==============================] - 0s 42us/step - loss: 0.2756 - acc: 0.8787
(437, 1)
(437, 1)
(437, 2)
(437, 1)
(437, 2)
(437, 3)
(437, 2)
(437, 5)
(437, 2)
(437, 7)
(437, 1)
(437, 9)
(437, 2)
(437, 9)
(437, 2)
(437, 11)
(437, 3)
(437, 1

437/437 [==============================] - 0s 44us/step - loss: 0.2749 - acc: 0.8787
(437, 1)
(437, 1)
(437, 2)
(437, 1)
(437, 2)
(437, 3)
(437, 2)
(437, 5)
(437, 2)
(437, 7)
(437, 1)
(437, 9)
(437, 2)
(437, 9)
(437, 2)
(437, 11)
(437, 3)
(437, 13)
(437, 4)
(437, 16)
(437, 3)
(437, 20)
(437, 2)
(437, 23)
(437, 2)
(437, 25)
(437, 2)
(437, 27)
(437, 2)
(437, 29)
Epoch 298/300
437/437 [==============================] - 0s 42us/step - loss: 0.2749 - acc: 0.8787
(437, 1)
(437, 1)
(437, 2)
(437, 1)
(437, 2)
(437, 3)
(437, 2)
(437, 5)
(437, 2)
(437, 7)
(437, 1)
(437, 9)
(437, 2)
(437, 9)
(437, 2)
(437, 11)
(437, 3)
(437, 13)
(437, 4)
(437, 16)
(437, 3)
(437, 20)
(437, 2)
(437, 23)
(437, 2)
(437, 25)
(437, 2)
(437, 27)
(437, 2)
(437, 29)
Epoch 299/300
437/437 [==============================] - 0s 42us/step - loss: 0.2748 - acc: 0.8810
(437, 1)
(437, 1)
(437, 2)
(437, 1)
(437, 2)
(437, 3)
(437, 2)
(437, 5)
(437, 2)
(437, 7)
(437, 1)
(437, 9)
(437, 2)
(437, 9)
(437, 2)
(437, 11)
(437, 3)
(437, 1

In [6]:
print(best_run)

{'batch_size': 64, 'choiceval': 'sgd', 'essai': 7.462695171578595, 'lr': 0.001, 'lr_1': 0.01, 'lr_2': 0.1}


In [7]:
trials.best_trial

{'state': 2,
 'tid': 0,
 'spec': None,
 'result': {'loss': -0.9198982835346472,
  'status': 'ok',
  'model': <keras.engine.training.Model at 0x102bdf438>,
  'predicted': array([3.34856912e-02, 7.00544183e-01, 9.21813662e-01, 9.88589119e-01,
         8.06145135e-02, 9.93286153e-01, 9.79592747e-01, 3.34856912e-02,
         6.06407517e-02, 1.55691329e-01, 1.75966253e-01, 9.94948492e-01,
         7.30141412e-02, 9.22825714e-01, 3.88741598e-02, 2.76624548e-02,
         9.96353930e-01, 1.01261367e-01, 9.24527573e-01, 3.34856912e-02,
         3.34856912e-02, 9.97173005e-01, 9.91749199e-01, 9.88377177e-02,
         1.75966253e-01, 9.84060281e-01, 9.65337126e-01, 5.80558116e-01,
         2.94994961e-01, 4.34549423e-01, 9.88589119e-01, 7.09865536e-01,
         5.18351346e-01, 9.79592747e-01, 2.57751691e-01, 9.41224259e-01,
         9.89959847e-01, 9.90092867e-01, 7.17586544e-01, 5.80558116e-01,
         9.67057332e-01, 8.02440009e-01, 4.26639938e-01, 9.94948492e-01,
         9.84060281e-01, 3.34

In [8]:
trials.results

[{'loss': -0.9198982835346472,
  'status': 'ok',
  'model': <keras.engine.training.Model at 0x102bdf438>,
  'predicted': array([3.34856912e-02, 7.00544183e-01, 9.21813662e-01, 9.88589119e-01,
         8.06145135e-02, 9.93286153e-01, 9.79592747e-01, 3.34856912e-02,
         6.06407517e-02, 1.55691329e-01, 1.75966253e-01, 9.94948492e-01,
         7.30141412e-02, 9.22825714e-01, 3.88741598e-02, 2.76624548e-02,
         9.96353930e-01, 1.01261367e-01, 9.24527573e-01, 3.34856912e-02,
         3.34856912e-02, 9.97173005e-01, 9.91749199e-01, 9.88377177e-02,
         1.75966253e-01, 9.84060281e-01, 9.65337126e-01, 5.80558116e-01,
         2.94994961e-01, 4.34549423e-01, 9.88589119e-01, 7.09865536e-01,
         5.18351346e-01, 9.79592747e-01, 2.57751691e-01, 9.41224259e-01,
         9.89959847e-01, 9.90092867e-01, 7.17586544e-01, 5.80558116e-01,
         9.67057332e-01, 8.02440009e-01, 4.26639938e-01, 9.94948492e-01,
         9.84060281e-01, 3.34856912e-02, 9.60384470e-01, 9.88589119e-01,
     

In [9]:
trials.best_trial['result']['predicted']

array([3.34856912e-02, 7.00544183e-01, 9.21813662e-01, 9.88589119e-01,
       8.06145135e-02, 9.93286153e-01, 9.79592747e-01, 3.34856912e-02,
       6.06407517e-02, 1.55691329e-01, 1.75966253e-01, 9.94948492e-01,
       7.30141412e-02, 9.22825714e-01, 3.88741598e-02, 2.76624548e-02,
       9.96353930e-01, 1.01261367e-01, 9.24527573e-01, 3.34856912e-02,
       3.34856912e-02, 9.97173005e-01, 9.91749199e-01, 9.88377177e-02,
       1.75966253e-01, 9.84060281e-01, 9.65337126e-01, 5.80558116e-01,
       2.94994961e-01, 4.34549423e-01, 9.88589119e-01, 7.09865536e-01,
       5.18351346e-01, 9.79592747e-01, 2.57751691e-01, 9.41224259e-01,
       9.89959847e-01, 9.90092867e-01, 7.17586544e-01, 5.80558116e-01,
       9.67057332e-01, 8.02440009e-01, 4.26639938e-01, 9.94948492e-01,
       9.84060281e-01, 3.34856912e-02, 9.60384470e-01, 9.88589119e-01,
       2.57751691e-01, 7.66609326e-02, 9.85623798e-01, 9.97035682e-01,
       9.38131254e-01, 9.84477063e-01, 3.66727564e-02, 2.08130309e-01,
      

In [10]:
x_quant,x_qual,x_qual_dummy,y,x_quant_test,x_qual_test,x_qual_dummy_test,y_test=data()


alpha = .95
y_pred = trials.best_trial['result']['predicted']
y_true = y_test

auc, auc_cov = delong_roc_variance(
    y_true,
    y_pred)

auc_std = np.sqrt(auc_cov)
lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)

ci = stats.norm.ppf(
    lower_upper_q,
    loc=auc,
    scale=auc_std)

ci[ci > 1] = 1

print('Gini:', 2*auc-1)
print('AUC COV:', auc_cov)
print('95% Gini CI:', 2*ci-1)

Gini: 0.919898283534647
AUC COV: 0.00014166472676970443
95% Gini CI: [0.87324213 0.96655444]
